## Loop on View  HST cutout images and Lupton colored image

- author : Sylvie Dagoret-Campagne
- creation date : 2025-05-30
- last update : 2025-05-30 

In [ ]:
import astroquery
print(astroquery.__version__)

https://stackoverflow.com/questions/59699193/how-to-obtain-2d-cutout-of-an-image-from-a-skycoord-position

In [ ]:
# For Angle conversion
import matplotlib.pyplot as plt
from astropy.coordinates import Angle
import astropy.units as u
from astropy.coordinates import SkyCoord
import pandas as pd
import numpy as np
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from astropy.nddata import Cutout2D
from astropy.visualization import simple_norm, ZScaleInterval,PercentileInterval,make_lupton_rgb

In [ ]:
from IPython.display import Image, display

## Configuration

### Target in ECDFS

- path to article : https://arxiv.org/pdf/1104.0931
- visual selection of the tiles : https://archive.stsci.edu/prepds/gems/browser.html
- path file download : https://archive.stsci.edu/pub/hlsp/gems/v_mk1/

In [ ]:
fig_table1 = "input_figs/table1_gemcandidates.png"
fig_table2 = "input_figs/table2_gemcandidates.png"

In [ ]:
Image(url= fig_table1,width=800)

In [ ]:
Image(url= fig_table2,width=500)

In [ ]:
#15422 44 03:32:38.21 –27:56:53.2 
ra1 = "03:32:38.21 hours"
dec1 = "-27:56:53.2 degrees"
tile1 = 44

#34244 94 03:32:06.45 –27:47:28.6 
ra2 = "03:32:06.45 hours"
dec2 = "-27:47:28.6 degrees"
tile2 = 94
# Je ne trouve pas cette tile ==> FindTileForCutoutGEM
tile2 = 32

#40173 35 03:33:19.45 –27:44:50.0 
ra3 = "03:33:19.45 hours"
dec3 = "-27:44:50.0 degrees"
tile3 = 35

#43242 45 03:31:55.35 –27:43:23.5 
ra4 = "03:31:55.35 hours"
dec4 = "-27:43:23.5 degrees"
tile4 = 45

#46446 47 03:31:35.94 –27:41:48.2 
ra5 = "03:31:35.94 hours"
dec5 = "-27:41:48.2 degrees"
tile5 = 47

#12589 03:31:24.89 −27:58:07.0
ra6 = "03:31:24.89 hours"
dec6 = "-27:58:07.0 degrees"
tile6 = 17

#43797 03:31:31.74 −27:43:00.8 
ra7 = "03:31:31.74 hours"
dec7 = "-27:43:00.8 degrees"
tile7 = 47

#28294 03:31:50.54 −27:50:28.4 
ra8 = "03:31:50.54 hours"
dec8 = "-27:50:28.4 degrees"
tile8 = 33

#36857 03:31:53.24 −27:46:18.9
ra9 = "03:31:53.24 hours"
dec9 = "-27:46:18.9 degrees"
tile9 = 38

#36714 03:32:59.78 −27:46:26.4 
ra10 = "03:32:59.78 hours"
dec10 = "-27:46:26.4 degrees"
tile10 = 37


In [ ]:
lsstcomcam_targets = {}
# high rank
lsstcomcam_targets["ECDFS_G15422"] = {"field_name": "GEMS-15422", "ra": 53.159208333333325, "dec": -27.94811111111111,"tile":tile1}
lsstcomcam_targets["ECDFS_G34244"] = {"field_name": "GEMS-34244", "ra": 53.02687499999999 , "dec": -27.79127777777778,"tile":tile2}
lsstcomcam_targets["ECDFS_G40173"] = {"field_name": "GEMS-40173", "ra": 53.33104166666666 , "dec": -27.747222222222224,"tile":tile3}
lsstcomcam_targets["ECDFS_G43242"] = {"field_name": "GEMS-43242", "ra": 52.980624999999996 , "dec": -27.72319444444444,"tile":tile4}
lsstcomcam_targets["ECDFS_G46446"] = {"field_name": "GEMS-46446", "ra": 52.89975 , "dec": -27.696722222222224,"tile":tile5}

# low rank
lsstcomcam_targets["ECDFS_G12589"] = {"field_name": "GEMS-12589", "ra": 52.85370833333333, "dec": -27.96861111111111,"tile":tile6}
lsstcomcam_targets["ECDFS_G43797"] = {"field_name": "GEMS-43797", "ra": 52.88224999999999, "dec": -27.71688888888889,"tile":tile7}

lsstcomcam_targets["ECDFS_G28294"] = {"field_name": "GEMS-28294", "ra": 52.960583333333325 , "dec": -27.84122222222222,"tile":tile8}
lsstcomcam_targets["ECDFS_G6857"] = {"field_name": "GEMS-6857", "ra": 52.97183333333333 , "dec": -27.771916666666666,"tile":tile9}
lsstcomcam_targets["ECDFS_G36714"] = {"field_name": "GEMS-36714", "ra": 53.249083333333324, "dec": -27.773999999999997,"tile":tile10}


In [ ]:
df = pd.DataFrame(lsstcomcam_targets).T

In [ ]:
# candidates
#key = "ECDFS_G15422"
#key = "ECDFS_G34244"
#key = "ECDFS_G40173"
#key= "ECDFS_G43242"
#key= "ECDFS_G46446"

# unknown
#key = "ECDFS_G12589"
#key = "ECDFS_G43797"
#key = "ECDFS_G28294"
#key = "ECDFS_G6857"
#key = "ECDFS_G36714"



## Config


In [ ]:
# select your bands
SELECT_VBAND = "v"
SELECT_ZBAND = "z"

In [ ]:
list_of_keys = ["ECDFS_G15422", "ECDFS_G34244","ECDFS_G40173","ECDFS_G43242","ECDFS_G46446","ECDFS_G12589","ECDFS_G43797","ECDFS_G28294","ECDFS_G6857","ECDFS_G36714"]

### Load image


🌈 Liste des stretch disponibles dans simple_norm
Tu peux utiliser ces chaînes de caractères comme valeur de stretch :
Valeur stretch	Description
- "linear"	Échelle linéaire simple (valeurs brutes entre min_cut et max_cut)
- "sqrt"	Échelle racine carrée – met en valeur les faibles intensités
- "power"	Échelle puissance (x^power)
- "log"	Échelle logarithmique – bon pour étendre la dynamique
- "asinh"	Échelle arcsinus hyperbolique – lisse, bien adaptée aux images astronomiques
- "sinh"	Hyperbolique sinus – moins courant
- "histeq"	Égalisation d'histogramme – utile pour répartir les contrastes

In [ ]:
def load_fits_image_with_norm(path):
    from astropy.io import fits
    from astropy.wcs import WCS
    with fits.open(path) as hdul:
        data = hdul[0].data
        wcs = WCS(hdul[0].header)
    vmin, vmax = ZScaleInterval().get_limits(data)
    #norm = simple_norm(data, stretch='asinh', min_cut=vmin, max_cut=vmax)
    norm = simple_norm(data, stretch='asinh', vmin=vmin, vmax=vmax)
    return data, wcs, norm

In [ ]:
def plot_multiband_panel(key, path_v, path_z):
    """
    Affiche une figure 2x2 : bande V, bande Z, RGB (V+Z), différence V-Z.
    key : nom de l'objet (titre)
    path_v : chemin image FITS bande V (peut être None ou invalide)
    path_z : chemin image FITS bande Z (doit exister)
    """

    # Tentative de chargement bande V
    has_v = True
    try:
        data_v, wcs_v, norm_v = load_fits_image_with_norm(path_v)
    except Exception:
        has_v = False
        data_v = None

    # Chargement bande Z (obligatoire)
    data_z, wcs_z, norm_z = load_fits_image_with_norm(path_z)

    # Créer la figure 2x2
    fig, axes = plt.subplots(2, 2, figsize=(12, 10),
                             subplot_kw={'projection': wcs_z})

    # Bande V
    ax1 = axes[0, 0]
    if has_v:
        ax1.imshow(data_v, origin='lower', cmap='gray', norm=norm_v)
        ax1.set_title(f'{key} V band')
    else:
        ax1.text(0.5, 0.5, 'V band image\nnot available', ha='center', va='center', fontsize=14)
        ax1.set_xticks([]); ax1.set_yticks([])
        ax1.set_title(f'{key} V band (missing)')
    ax1.set_xlabel('RA')
    ax1.set_ylabel('Dec')

    # Bande Z
    ax2 = axes[0, 1]
    ax2.imshow(data_z, origin='lower', cmap='gray', norm=norm_z)
    ax2.set_title(f'{key} Z band')
    ax2.set_xlabel('RA')
    ax2.set_ylabel('Dec')

    # RGB Lupton (Z=R, V=B, G=moyenne)
    ax3 = axes[1, 0]
    if has_v:
        G = 0.5 * (data_v + data_z)
        #rgb = make_lupton_rgb(R=data_z, G=G, B=data_v, stretch=0.5, Q=10)
        rgb = make_lupton_rgb(image_r=data_z, image_g=G, image_b=data_v, stretch=0.02, Q=10)
        ax3.imshow(rgb, origin='lower')
        ax3.set_title(f'{key} RGB (B=v, R=z)')
    else:
        ax3.text(0.5, 0.5, 'RGB combination\nnot available', ha='center', va='center', fontsize=14)
        ax3.set_xticks([]); ax3.set_yticks([])
        ax3.set_title(f'{key} RGB (missing)')
    ax3.set_xlabel('RA')
    ax3.set_ylabel('Dec')

    # Différence V - Z
    ax4 = axes[1, 1]
    if has_v:
        #diff = data_v - data_z
        #diff = -2.5*np.log10(data_v) + 2.5*np.log10(data_z)
        eps = 1e-10
        diff = 2.5 * np.log10((data_z + eps) / (data_v + eps))
        norm_diff = simple_norm(diff, stretch='linear', percent=97)
        #im = ax4.imshow(diff, origin='lower', cmap='seismic', norm=norm_diff)
        im = ax4.imshow(diff, origin='lower', cmap='coolwarm', norm=norm_diff)
        ax4.set_title(f'{key} V - Z')

        # Ajouter une colorbar interne
        cax = ax4.inset_axes([1.02, 0.1, 0.02, 0.8])  # [left, bottom, width, height]
        fig.colorbar(im, cax=cax, orientation='vertical',label='V - Z (mag)')
    else:
        ax4.text(0.5, 0.5, 'V - Z not available', ha='center', va='center', fontsize=14)
        ax4.set_xticks([]); ax4.set_yticks([])
        ax4.set_title(f'{key} V - Z (missing)')
    ax4.set_xlabel('RA')
    ax4.set_ylabel('Dec')

    # Mise en page
    plt.tight_layout()
    plt.show()

## Loop on targets

In [ ]:
for ik,key in enumerate(list_of_keys):


    the_target = lsstcomcam_targets[key]
    target_ra = the_target["ra"]
    target_dec = the_target["dec"]
    target_name = the_target["field_name"]
    tile_num = the_target["tile"]
    coord = SkyCoord(ra=target_ra, dec=target_dec, unit=(u.deg, u.deg))

    target_title = (
        the_target["field_name"] + f" (ra,dec) = ({target_ra:.2f},{target_dec:.2f}) "
    )

    cutout_inputpath_v = f"cutout_gems_{key}_b{SELECT_VBAND}.fits"
    cutout_inputpath_z = f"cutout_gems_{key}_b{SELECT_ZBAND}.fits"

    plot_multiband_panel(key=key,path_v=cutout_inputpath_v,path_z=cutout_inputpath_z)


    